In [11]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

import numpy as np
import os
import copy
import datetime

# torch.set_printoptions(profile="full")
# np.set_printoptions(threshold=np.inf)


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [13]:
# parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 100

In [14]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [15]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [16]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 25, kernel_size=3),
            torch.nn.BatchNorm2d(25),
            torch.nn.ReLU(inplace=True)
        )
 
        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
 
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(25, 50, kernel_size=3),
            torch.nn.BatchNorm2d(50),
            torch.nn.ReLU(inplace=True)
        )
 
        self.layer4 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
 
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(50 * 5 * 5, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 10)
        )
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = MyNet().to(device)
print(model)

MyNet(
  (layer1): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1250, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1024, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [17]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
# model.load_state_dict(torch.load("./myfed_normal_save/model18-32-05.pth")["state_dict"])
dir = os.listdir("./merge_save")
modelnames = []
for mt in dir:
    if(mt[0:len("merge_attack")] == "merge_attack"):
        modelnames.append(mt)
print(modelnames)
modelset = []
for mn in modelnames:
    modelset.append([model.load_state_dict(torch.load("./merge_save/"+mn)["state_dict"]),mn])
# print(dir)
savepath = "./metric/vr_metric="

['merge_attack0normal20_06-05--17-27-55.pth', 'merge_attack0normal20_06-05--17-28-09.pth', 'merge_attack0normal20_06-05--17-28-20.pth', 'merge_attack0normal20_06-05--17-28-32.pth', 'merge_attack0normal20_06-05--17-28-38.pth', 'merge_attack0normal20_06-05--17-29-32.pth', 'merge_attack0normal20_06-05--17-29-38.pth', 'merge_attack0normal20_06-05--17-29-40.pth', 'merge_attack0normal20_06-05--17-29-43.pth', 'merge_attack0normal20_06-05--17-29-46.pth', 'merge_attack0normal20_06-05--17-29-48.pth', 'merge_attack0normal20_06-05--17-29-51.pth', 'merge_attack0normal20_06-05--17-52-30.pth', 'merge_attack0normal20_06-05--17-52-34.pth', 'merge_attack0normal20_06-05--17-52-36.pth', 'merge_attack0normal20_06-05--17-52-39.pth', 'merge_attack0normal20_06-05--17-52-43.pth', 'merge_attack0normal20_06-05--17-52-45.pth', 'merge_attack0normal20_06-05--17-52-48.pth', 'merge_attack0normal20_06-05--17-52-52.pth', 'merge_attack0normal20_06-05--17-52-55.pth', 'merge_attack0normal20_06-05--17-52-57.pth', 'merge_at

In [19]:
for mode, modelname in modelset:
    parm = {}
    for name,parameters in model.named_parameters():
        parm[name]=parameters
        # print(parm[name])
    tensors = []
    model.eval()
    with torch.no_grad():

        t=10
        for i in range(0,t):
            r = random.randint(0, (batch_size) - 1)

            for X_test, Y_test in test_loader:
                X_test = X_test.to(device)
                Y_test = Y_test.to(device)
            

            X_single_data = X_test[r]
            Y_single_data = Y_test[r]

            # print(X_single_data)
            
            layerP = model.layer1(torch.unsqueeze(X_single_data,dim=0))
            # print("1",layerP)
            layerP = model.layer2(layerP)
            # print("2",layerP)
            layerP = model.layer3(layerP)
            # print("3",layerP)
            layerP = model.layer4(layerP)
            layerP = layerP.detach().squeeze().reshape(-1,1250)

            tensor1 = torch.mm(layerP, parm['fc.0.weight'].data.permute(1,0)) + parm['fc.0.bias']
            tensor1 = torch.mm(tensor1, parm['fc.2.weight'].data.permute(1,0)) + parm['fc.2.bias']
            
            tensors.append(tensor1.cpu().detach().numpy()[0])

    nb = np.linspace(0,0,tensors[0].shape[0])
    for i in tensors:
        nb += i
    nb /= t

    ns = np.linspace(0,0,tensors[0].shape[0])
    vp = np.linspace(0,0,tensors[0].shape[0] ** 2).reshape(tensors[0].shape[0],tensors[0].shape[0])

    for i in range(0,len(ns)):
        for nps in tensors:
            ns[i] += nps[i] ** 2
        ns[i] = ns[i] ** 0.5

    for i in range(0,len(ns)):
        for j in range(i+1,len(ns)):
            for nps in tensors:
                vp[i,j] += (nps[i] - nb[i]) * (nps[j] - nb[j])
            vp[i,j] /= (ns[i] * ns[j])

    dt = datetime.datetime.now()
    # np.savetxt("./misc/vr_metric"+dt.strftime("%d--%H-%M-%S")+".dat", vp+vp.T, fmt="%1.5f")
    np.savetxt(savepath+modelname+dt.strftime("%m-%d--%H-%M-%S")+".dat", vp+vp.T, fmt="%1.5f")